In [58]:
using JuMP, Gurobi, Base.Test, Cbc

In [3]:
import JuMPIndicators

JuMPIndicators

In [50]:
reload("JuMPIndicators")
Ifs = JuMPIndicators

JuMPIndicators

In [51]:
m = Model()
@variable m 1 <= x <= 3
e = 2 * x + 1
Ifs.upperbound(e)

7.0

In [33]:
function update(x)
    Ifs.@disjunction if x <= 0
        1
    else
        -1
    end
end

update (generic function with 1 method)

In [59]:
        m = Model(solver=CbcSolver())
        @variable m -0.5 <= x <= 0.5

        ys = [x]
        for i in 1:3
            push!(ys, update(ys[end]))
        end

        @objective m Max sum(ys)
        display(m)
        solve(m)
        getvalue.(ys)

Maximization problem with:
 * 18 linear constraints
 * 7 variables: 3 binary
Solver is CbcMathProg

4-element Array{Float64,1}:
  0.0
  1.0
 -1.0
  1.0

In [34]:
x = 0.5
ys = [x]
for i in 1:3
    push!(ys, update(ys[end]))
end
ys

4-element Array{Float64,1}:
  0.5
 -1.0
  1.0
 -1.0

In [35]:
m = Model(solver=GurobiSolver())
@variable m -5 <= x <= 5

ys = [x]
for i in 1:3
    push!(ys, update(ys[end]))
end

@objective m Min sum((ys .- 0.5).^2)
display(m)
solve(m)
getvalue.(ys)

Minimization problem with:
 * 18 linear constraints
 * 7 variables: 3 binary
Solver is Gurobi

Optimize a model with 18 rows, 7 columns and 30 nonzeros
Model has 4 quadratic objective terms
Variable types: 4 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 13 rows and 2 columns
Presolve time: 0.00s
Presolved: 5 rows, 5 columns, 10 nonzeros
Presolved model has 4 quadratic objective terms
Variable types: 1 continuous, 4 integer (1 binary)
Found heuristic solution: objective 2.0000000

Root relaxation: objective -3.333333e-01, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.33333    0    4    2.00000   -0.33333   117%     -    0s

Explored 0 nodes (5 simplex iterations) in 0.00 seconds
Thread count was 8 (of 8 available processors)

Solution co

4-element Array{Float64,1}:
  0.0
  1.0
 -1.0
  1.0

In [37]:
m = Model(solver=GurobiSolver())
@variable m x
setvalue(x, -1)

ys = [x]
for i in 1:3
    push!(ys, update(ys[end]))
end

# Ifs.setup_disjunctions!(m, [x] => [-1])

@objective m Min sum((ys .- 0.5).^2)
solve(m)
getvalue.(ys)

Optimize a model with 6 rows, 4 columns and 6 nonzeros
Model has 4 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds
Optimal objective 2.00000000e+00


4-element Array{Float64,1}:
 -0.0
  1.0
 -1.0
  1.0

In [39]:
m = Model(solver=GurobiSolver())
@variable m x
setvalue(x, 1)

ys = [x]
for i in 1:3
    push!(ys, update(ys[end]))
end

@objective m Min sum((ys .- 0.5).^2)
solve(m)
getvalue.(ys)

Optimize a model with 6 rows, 4 columns and 6 nonzeros
Model has 4 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.00 seconds
Optimal objective 3.75000000e+00


4-element Array{Float64,1}:
  0.5
 -1.0
  1.0
 -1.0

In [360]:
m = Model(solver=GurobiSolver())
@variable m -5 <= x <= 5

y = update(x)

Ifs.setup_disjunctions!(m)

display(m)

Feasibility problem with:
 * 6 linear constraints
 * 3 variables: 1 binary
Solver is Gurobi

In [275]:
m = Model(solver=GurobiSolver())
@variable m -5 <= x <= 5

y = update(x)

Ifs.setup_disjunctions!(m, [x] => [3])

display(m)

Feasibility problem with:
 * 2 linear constraints
 * 2 variables
Solver is Gurobi

In [194]:
m = Model()
@variable m x
update(x)

y

In [195]:
m

Feasibility problem with:
 * 6 linear constraints
 * 3 variables: 1 binary
Solver is default solver

In [191]:
x = 0.5
ys = [x]
for i in 1:3
    push!(ys, update(ys[end]))
end
ys

4-element Array{Float64,1}:
  0.5
 -1.0
  1.0
 -1.0

In [192]:
m = Model(solver=GurobiSolver())
@variable m x
@constraint m x == 0.5

ys = [x]
for i in 1:3
    push!(ys, update(ys[end]))
end

solve(m)
getvalue.(ys)

Optimize a model with 19 rows, 7 columns and 37 nonzeros
Variable types: 4 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 1e+01]
Presolve removed 19 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


4-element Array{Float64,1}:
  0.5
 -1.0
  1.0
 -1.0

In [182]:
@code_warntype update(0.5)

Variables:
  #self#::#update
  x::Float64
  cond::Any
  comp::Any
  m::Any
  y::Any
  ##1915::Any
  fy::Float64

Body:
  begin 
      # meta: location In[170] # line 37:
      SSAValue(12) = x::Float64
      goto 6 # line 101:
      6:  # line 110:
      $(Expr(:inbounds, false))
      # meta: location float.jl <= 495
      fy::Float64 = (Base.sitofp)(Float64, 0)::Float64
      # meta: pop location
      $(Expr(:inbounds, :pop))
      unless (Base.or_int)((Base.lt_float)(x::Float64, fy::Float64)::Bool, (Base.and_int)((Base.and_int)((Base.eq_float)(x::Float64, fy::Float64)::Bool, (Base.lt_float)(fy::Float64, 9.223372036854776e18)::Bool)::Bool, (Base.sle_int)((Base.fptosi)(Int64, fy::Float64)::Int64, 0)::Bool)::Bool)::Bool goto 15
      return 1
      15: 
      # meta: pop location
      return -1
  end::Int64


In [164]:
m = Model(solver=GurobiSolver())
@variable m x

y = @Ifs.switch if x <= 0
    5
else
    3
end
@constraint m x == 1


@objective m Min x^2 + y^2
solve(m)
getvalue(y)

Optimize a model with 7 rows, 3 columns and 13 nonzeros
Model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 7 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 10 
Pool objective bound 10

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


3.0000000000000053

In [162]:
m = Model(solver=GurobiSolver())
@variable m x

y = @Ifs.switch if x <= 0
    5
end
@constraint m x == -1


@objective m Min x^2 + y^2
solve(m)
getvalue(y)

Optimize a model with 5 rows, 3 columns and 9 nonzeros
Model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 26 
Pool objective bound 26

Optimal solution found (tolerance 1.00e-04)
Best objective 2.600000000000e+01, best bound 2.600000000000e+01, gap 0.0000%


5.000000000000006

In [76]:
m = Model(solver=GurobiSolver())
@variable m x
@variable m y
@objective m Min x^2 + y^2

Ifs.implies(m, @Ifs.condition(x <= 0 => y == 5))
@constraint m x == 1
solve(m)
getvalue(y)

Optimize a model with 5 rows, 3 columns and 9 nonzeros
Model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1 
Pool objective bound 1

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%


-0.0

In [55]:
c = @Ifs.condition(x == 2 => 3)

3 if x == 2

In [ ]:
m = Model(solver=GurobiSolver())
@variable m x
y = Ifs.exclusive(m, @Ifs.condition(x <= 0))

In [41]:
m = Model(solver=GurobiSolver())
@variable m x



@variable m z Bin
Ifs.implies(m, z, @Ifs.condition x == 2)
@constraint m z == 0
solve(m)
getvalue(x)

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Variable types: 1 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 1e+01]
Found heuristic solution: objective 0

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


0.0

In [39]:
m

Feasibility problem with:
 * 2 linear constraints
 * 2 variables: 1 binary
Solver is Gurobi

In [28]:
m = Model()
@variable m x
c = Ifs.@condition x == 5

x == 5

In [16]:
m = Model(solver=GurobiSolver())
@variable m x

options = (
    ConditionalValue(
        Condition{:(<=)}(x, 0),
        -x),
    ConditionalValue(
        Condition{:(<=)}(-x, -0),
        zero(x))
        )

(-x if x <= 0, 0 if -x <= 0)

In [33]:
function dynamics(x)
    f = if isa(x, JuMP.AbstractJuMPScalar)
        M = 10
        model = getmodel(x)
        z = @variable(model, category=:Bin, basename="z")
        @constraint(model, x <= 0 + M * (1 - z))
        @constraint(model, x >= 0 - M * z)
        y = @variable(model, basename="y")
        @constraint(model, y <= -x + M * (1 - z))
        @constraint(model, y >= -x - M * (1 - z))
        @constraint(model, y <= zero(x) + M * z)
        @constraint(model, y >= zero(x) - M * z)
        y
    else
        if x <= 0
            -x
        else
            zero(x)
        end
    end
    y = x + 0.1 * f
end

dynamics (generic function with 1 method)

In [36]:
m = Model(solver=GurobiSolver())
@variable m x
@objective m Min y^2
@constraint m x == -1
y = dynamics(x)
ys = [y]
for i in 1:5
    push!(ys, dynamics(ys[end]))
end

In [37]:
solve(m)
getvalue(x), getvalue.(ys)

LoadError: [91mUndefVarError: VariableNotOwnedException not defined[39m

In [4]:
x <= 1

LoadError: [91mMethodError: no method matching isless(::Int64, ::JuMP.Variable)[0m
Closest candidates are:
  isless(::Real, [91m::AbstractFloat[39m) at operators.jl:97
  isless(::Real, [91m::ForwardDiff.Dual[39m) at /Users/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/ForwardDiff/src/dual.jl:161
  isless(::Real, [91m::Real[39m) at operators.jl:266[39m

In [5]:
expr = :(
if x <= 1
    k * x
else
    0
end
)

:(if x <= 1 # In[5], line 3:
        k * x
    else  # In[5], line 5:
        0
    end)

In [8]:
supertype(JuMP.Variable)

JuMP.AbstractJuMPScalar

In [9]:
supertype(JuMP.AffExpr)

JuMP.AbstractJuMPScalar

In [11]:
if isa(x, JuMP.AbstractJuMPScalar)
    M = 10
    z = @variable(x.m, category=:Bin, basename="z")
    @constraint(x.m, x <= 1 + M * (1 - z))
    y = @variable(x.m, basename="y")
    @constraint(x.m, y <= 1 + M * (1 - z))
    @constraint(x.m, y >= 1 - M * (1 - z))
    @constraint(x.m, y <= 2 + M * z)
    @constraint(x.m, y >= 2 - M * z)
    y
end

y

In [7]:
dump(:(x <= 1 ? 1 : 2))

Expr
  head: Symbol if
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol <=
        2: Symbol x
        3: Int64 1
      typ: Any
    2: Int64 1
    3: Int64 2
  typ: Any


In [6]:
dump(expr)

Expr
  head: Symbol if
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol <=
        2: Symbol x
        3: Int64 1
      typ: Any
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: Expr
          head: Symbol line
          args: Array{Any}((2,))
            1: Int64 3
            2: Symbol In[5]
          typ: Any
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Symbol k
            3: Symbol x
          typ: Any
      typ: Any
    3: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: Expr
          head: Symbol line
          args: Array{Any}((2,))
            1: Int64 5
            2: Symbol In[5]
          typ: Any
        2: Int64 0
      typ: Any
  typ: Any
